In [1]:
import ee
ee.Initialize()
from iteru import *

In [2]:
Map = Map()
Map

Map(center=[27, 31], controls=(AttributionControl(options=['position', 'prefix'], position='bottomright'), Sea…

In [224]:
start_date = '2015-08-03'
end_date = '2022-01-01'
frequency = 30
aoi = Map.aoi
days = ee.List.sequence(2015,2022,1)

In [215]:
col1 = ee.ImageCollection("COPERNICUS/S2_SR")
col2 = ee.ImageCollection("COPERNICUS/S2")

In [216]:
def rename_bands(img):
    return img.select(["B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B11", "B12", "QA60"],
                      ["Blue","Green","Red","Red Edge 1","Red Edge 2","Red Edge 3","NIR","Red Edge 4","SWIR1","SWIR2","QA60"])

In [217]:
col1= col1.map(rename_bands)
col2= col2.map(rename_bands)

In [218]:
col = col1.merge(col2)

In [230]:
def get_days(day):
    start_day = ee.Date(start_date)
    end_day =  start_day.advance(1,'month')
    return col.filterBounds(aoi).filterDate(start_day,end_day).reduce(ee.Reducer.median())

In [231]:
images = days.map(get_days)

In [232]:
collection = ee.ImageCollection.fromImages(images)
collection.size().getInfo()

8

In [202]:
videoArgs = {
  'dimensions': 768,
  'region': aoi,
  'framesPerSecond': 5,
  'crs': 'EPSG:3857',
  'min': 0,
  'max': 5000,
  'bands':['Red_median','Green_median','Blue_median']
}

In [233]:
print(collection.getVideoThumbURL(videoArgs))

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/f183a3efee482bd8b1cfc0da3961bfdb-ed9e878417a9513c29d3ad1b909a315e:getPixels
